In [ ]:
from IPython.display import Image
import ee
ee.Initialize()
import sys 
sys.path.append('/rheil/notebooks/pygee')
import pandas as pd
import numpy as np
import simulation
import importlib
import gee_random
importlib.reload(gee_random)
importlib.reload(simulation)
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns


In [ ]:
chile_comunas = ee.FeatureCollection('ft:1MIVtGXFEu3ONr87OPP7JV1_vCs9W3sJEuZDuzkzb')
chile_comunas = chile_comunas.filter(ee.Filter.inList('id_1', [15, 13, 8, 12, 6, 3, 10, 9]))

# regions = ee.FeatureCollection('ft:1RfoT4UoTohqmJJrNkMT5Jyp95SahGk5f9V355SnH')
# regions = regions.filter(ee.Filter.inList('COD_REG', ['05', '06', '07', '08', '09', '10', '13', '14']))
# reg = regions.filterMetadata('COD_REG', 'equals', '08')
coefs_csv = '/rheil/notebooks/data/pooled_coefs.csv'
coefs_se_csv = '/rheil/notebooks/data/pooled_se.csv'
asset_dir = 'users/rheilmayr/chile_sim/input/'
out_dir = 'users/rheilmayr/chile_sim/simulation/'

sample_region = ee.Algorithms.GeometryConstructors.Polygon([[-73.25,-37.4], [-73.25,-37.6], 
    [-73,-37.6], [-73,-37.4], [-73.25,-37.4]])
sample_region = ee.Feature(sample_region)

In [ ]:
n = 3
np.random.seed(1235)

In [ ]:
base_dict ={'agrent': asset_dir + 'agrent_01',
            'forrent': asset_dir + 'forrent',
            'south': asset_dir + 'south',
            'central': asset_dir + 'central',
            'noprot': asset_dir + 'noprot',
            'luq2': asset_dir + 'luq2',
            'luq3': asset_dir + 'luq3',
            'mask': asset_dir + 'mask'}

map_dicts = {key: base_dict.copy() for key in ['01_sub', '01_ns', '11_rs', '11_ns', '11_sub']}
map_dicts['01_sub'].update({'plantrent': asset_dir + 'plantrent_01',
                            'olu': asset_dir + 'olu_1986'})
map_dicts['01_ns'].update({'plantrent': asset_dir + 'plantrent_01_ns',
                           'olu': asset_dir + 'olu_1986'})
map_dicts['11_rs'].update({'plantrent': asset_dir + 'plantrent_11_rs'})
map_dicts['11_ns'].update({'plantrent': asset_dir + 'plantrent_11_ns'})
map_dicts['11_sub'].update({'plantrent': asset_dir + 'plantrent_11'})

In [ ]:
class mask_updater:
    def __init__(self, mask):
        self.mask = mask
    def __call__(self, img):
        img = img.updateMask(self.mask)
        return img

def sim_flow(input_ic, coefs_csv, coefs_se_csv, n):
    template = ee.Image(input_ic.first()).select('olu')
    rand_imgs = gee_random.runiform_rasters(template, n)
    sim_mapper = simulation.sim_mapper(coefs_csv, coefs_se_csv)
    pr_ic = input_ic.map(sim_mapper)
    cum_pr_ic = pr_ic.map(simulation.gen_cum_img)
    cum_pr_ic = cum_pr_ic.combine(rand_imgs.select('Ran_uniform'))
    clas_ic = cum_pr_ic.map(simulation.assign_clas)
    update_mask = mask_updater(ee.Image(input_ic.first()).select('mask').eq(0))
    clas_ic = clas_ic.map(update_mask)
    pr_ic = pr_ic.map(update_mask)
    cum_pr_ic = cum_pr_ic.map(update_mask)
    return pr_ic, cum_pr_ic, clas_ic

class band_renamer:
    def __init__(self, orig_bandnames, new_bandnames):
        self.orig_bandnames = orig_bandnames
        self.new_bandnames = new_bandnames
    def __call__(self, img):
        img = img.select(self.orig_bandnames, self.new_bandnames)
        return img

def band_aggregator(new_band, full_img):
    """
    Converts an image collection into a single multiband image.
    To be used through iterate on an image collection.
    """
    full_img = ee.Image(full_img).addBands(new_band)
    return full_img

rename_olu = band_renamer(['clas'], ['olu'])

# def rename_clas(img):
#     img = img.select(['clas'], ['olu'])
#     return img

Run subsidized simulation

In [ ]:
input_ic_p1 = simulation.dict_to_input_ic(map_dicts['01_ns'], n)
pr_ic_p1, cum_pr_ic_p1, clas_ic_p1 = sim_flow(input_ic_p1, coefs_csv, coefs_se_csv, n)
input_ic_p2 = simulation.dict_to_input_ic(map_dicts['11_ns'], n)
clas_ic_p1 = clas_ic_p1.map(rename_olu)
input_ic_p2 = input_ic_p2.combine(clas_ic_p1)
pr_ic_p2, cum_pr_ic_p2, clas_ic_p2 = sim_flow(input_ic_p2, coefs_csv, coefs_se_csv, n)

In [ ]:
clas_img = clas_ic_p2.iterate(band_aggregator, ee.Image())
img_bands = ['clas_' + str(i+1) for i in range(n-1)]
img_bands.extend(['clas'])
clas_img = ee.Image(clas_img).select(img_bands)
def sum_clas_area(img, clas, fc):
    binary_img = img.eq(clas)
    clasarea_img = binary_img.multiply(ee.Image.pixelArea().divide(1e7))
    new_bands = ['c' + str(clas) + '_' + str(i) for i in range(n)]
    clasarea_img = clasarea_img.select(img_bands, new_bands)
    fc = clasarea_img.reduceRegions(collection = fc, scale = 30, reducer = ee.Reducer.sum())
    return fc

for c in [1, 3, 5, 19]:
    chile_comunas = sum_clas_area(clas_img, c, chile_comunas)

In [ ]:
task = ee.batch.Export.table.toDrive(collection = chile_comunas, description = 'lusim_n3_ns', folder = 'chile/sim/', fileFormat = 'csv')
task.start()

In [ ]:
results_dir = '/rheil/notebooks/data/gee_results/'
results_dict  = {key: results_dir + 'lusim_n6_' + key + '.csv' for key in ['sub', 'ns', 'rs']}
results_df_dict = {}
for key, result in results_dict.items():
    results_df_dict[key] = pd.read_csv(result)
results_df = pd.concat(results_df_dict)
results_df = results_df.set_index('COD_REG', append = True)
keep_cols = [col for col in results_df.columns if col[0] == 'c']
results_df = results_df[keep_cols]
clas = [col[1:col.rfind('_')] for col in results_df.columns]
run = [col[col.rfind('_')+1:] for col in results_df.columns]
results_df = results_df.T
results_df['clas'] = clas
results_df['run'] = run
results_df = results_df.set_index(['clas', 'run'])

In [ ]:
results_df.sum(axis = 1, level = 0).loc['1']['ns']

In [ ]:
ax = sns.kdeplot(results_df.sum(axis = 1, level = 0).loc['1']['ns'], results_df.sum(axis = 1, level = 0).loc['3']['ns'],
                 cmap="Reds", shade=True, shade_lowest=False)
ax = sns.kdeplot(results_df.sum(axis = 1, level = 0).loc['1']['sub'], results_df.sum(axis = 1, level = 0).loc['3']['sub'],
                 cmap="Greens", shade=True, shade_lowest=False)
ax = sns.kdeplot(results_df.sum(axis = 1, level = 0).loc['1']['rs'], results_df.sum(axis = 1, level = 0).loc['3']['rs'],
                 cmap="Blues", shade=True, shade_lowest=False)

In [ ]:
task = ee.batch.Export.image.toAsset(image = ee.Image(cum_pr_ic_p2.first()).clip(sample_region), description = 'cum_pr_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/cum_pr_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(pr_ic_p2.first()).clip(sample_region), description = 'pr_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/pr_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(clas_ic_p2.first()).clip(sample_region), description = 'clas_img_clip_2', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/clas_clip_2', 
                                     scale = 30, maxPixels = 1e10)
task.start()
# task = ee.batch.Export.image.toAsset(image = ee.Image(cum_pr_ic_p1.first()).clip(sample_region), description = 'cum_pr_img_clip_1', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/cum_pr_clip_1', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()
# task = ee.batch.Export.image.toAsset(image = ee.Image(pr_ic_p1.first()).clip(sample_region), description = 'pr_img_clip_1', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/pr_clip_1', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()
# task = ee.batch.Export.image.toAsset(image = ee.Image(clas_ic_p1.first()).clip(sample_region), description = 'clas_img_clip_1', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/clas_clip_1', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()

In [ ]:
task = ee.batch.Export.image.toAsset(image = ee.Image(cum_pr_ic_p2.first()), description = 'cum_pr_img', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/cum_pr', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(pr_ic_p2.first()), description = 'pr_img', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/pr', 
                                     scale = 30, maxPixels = 1e10)
task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(clas_ic_p2.first()), description = 'clas_img', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/clas', 
                                     scale = 30, maxPixels = 1e10)
task.start()

In [ ]:
# task = ee.batch.Export.image.toAsset(image = ee.Image(input_ic_p2.first()).clip(sample_region), description = 'input_p2', 
#                                      assetId = 'users/rheilmayr/chile_sim/simulations/inputs_p2', 
#                                      scale = 30, maxPixels = 1e10)
# task.start()
task = ee.batch.Export.image.toAsset(image = ee.Image(input_ic_p1.first()).clip(sample_region), description = 'input_p1', 
                                     assetId = 'users/rheilmayr/chile_sim/simulations/inputs_p1', 
                                     scale = 30, maxPixels = 1e10)
task.start()

In [ ]:
results_df.sum(axis = 1)

In [ ]:
luq2 = ee.Image(asset_dir + 'luq2')
luq3 = ee.Image(asset_dir + 'luq3')
noprot = ee.Image(asset_dir + 'noprot')
central = ee.Image(asset_dir + 'central')
south = ee.Image(asset_dir + 'south')
agrent01 = ee.Image(asset_dir + 'agrent_01')
agrent11 = ee.Image(asset_dir + 'agrent_11')
plantrent01 = ee.Image(asset_dir + 'plantrent_01')
plantrent11 = ee.Image(asset_dir + 'plantrent_11')
forrent = ee.Image(asset_dir + 'forrent')
olu_1986 = ee.Image(asset_dir + 'olu_1986')
mask = (luq2.eq(0).Or(luq2.eq(1)))\
            .And(luq3.eq(0).Or(luq3.eq(1)))\
            .And(noprot.eq(0).Or(noprot.eq(1)))\
            .And(central.eq(0).Or(central.eq(1)))\
            .And(south.eq(0).Or(south.eq(1)))\
            .And(agrent01.gt(-3.4e38))\
            .And(agrent11.gt(-3.4e38))\
            .And(plantrent01.gt(-3.4e38))\
            .And(plantrent11.gt(-3.4e38))\
            .And(forrent.gt(-3.4e38))\
            .And(olu_1986.eq(1).Or(olu_1986.eq(3)).Or(olu_1986.eq(5)).Or(olu_1986.eq(19)))
        
task = ee.batch.Export.image.toAsset(image = mask, description = 'mask', 
                                     assetId = 'users/rheilmayr/chile_sim/inputs/mask_new', 
                                     scale = 30, maxPixels = 1e10)
task.start()


In [ ]:
# var multiMax = function(feature) {
#   // Reduce to mean value the input polygon for each image
#   var means = collection.map(function(image) {
#       var a = image.reduceRegion({
#         reducer: ee.Reducer.mean(),
#         geometry: feature.geometry(),
#         scale: 250
#       });
#       // Extract the reduced value from the returned dictionary and add it 
#       // to the image as property 'mean'.
#       image = image.set('mean', a.get('EVI'));
#       return image;
#   });

class multiMapAreas:
    def __init__(self, ic):
        self.ic = ic
    def sum_features(self, feature):
        self.feature = feature
        areas_ic = self.ic.map(self.sum_img)
        return areas_ic
    def sum_img(self, img):
        areas = img.reduceRegion(reducer = ee.Reducer.sum(),
                                 geometry = self.feature.geometry(),
                                 scale = 500)
        clases = ['for', 'plant', 'shrub', 'ag']
        clas_dict = {}
        for clas in clases:
            clas_dict[clas] = areas.get(clas)
        img = img.set(clas_dict)
        return img
    
# def sum_ic(feature):
#     def sum_img(img):
#         areas = img.reduceRegion(reducer = ee.Reducer.sum(),
#                                  geometry = feature.geometry(),
#                                  scale = 500)
#         return areas
#     areas = clas_ic.map(sum_img)
#     clases = ['for', 'plant', 'shrub', 'ag']
#     clas_dict = {}
#     for clas in clases:
#         clas_dict[clas] = areas.get(clas)
#     img = img.set(clas_dict)
#     return img

# clas_ic = clas_ic.map(sum_freqs)
# clas_info = clas_ic.getInfo()

In [ ]:
multiMapper = multiMapAreas(clas_ic)
areas_ic = regions.map(multiMapper.sum_features)
areas_ic.getInfo()

In [ ]:
img = ee.Image(clas_ic.first())
feature = ee.Feature(reg.first())

def sum_img(img):
    areas = img.reduceRegion(reducer = ee.Reducer.sum(),
                             geometry = feature.geometry(),
                             scale = 500)
    clases = ['for', 'plant', 'shrub', 'ag']
    clas_dict = {}
    img_id = img.get('system:index')
    for clas in clases:
        clas_dict['r' + img_id + '_' + clas] = areas.get(clas)
    img = img.set(clas_dict)
    return img
sum_img(img).getInfo()

In [ ]:
clas_img = ee.Image(clas_ic.first())
area_img = clas_img.multiply(ee.Image.pixelArea().divide(10000000))
def add_areas(feature):
    clases = ['for', 'plant', 'shrub', 'ag']
    areas = area_img.reduceRegion(reducer = ee.Reducer.sum(), 
                                    geometry = feature.geometry(), 
                                    scale = 1000, 
                                    maxPixels = 1e10)
    clas_dict = {}
    for clas in clases:
        clas_dict[clas] = areas.get(clas)
    feature = feature.set(clas_dict)
    return feature

out_regions = regions.map(add_areas)

In [ ]:
out_dict = {}
for i, region_dict in enumerate(test_info['features']):
    out_dict[i] = region_dict['properties']
out_df = pd.DataFrame(out_dict)
out_df = out_df.T
out_df = out_df.set_index('COD_REG').sort_index()
out_df[['for', 'plant', 'shrub', 'ag']]

In [ ]:
stats = clas_img.reduceRegion(reducer = ee.Reducer.frequencyHistogram(), 
                                      geometry = reg, 
                                      scale = 100, 
                                      maxPixels = 9e9)
stats.getInfo()

In [ ]:
Image(url=clas_img.select('for').getThumbUrl({'min':0, 'max':1}))

In [ ]:
#mask = map_dict['olu'].eq(0).Or(map_dict['olu'].eq(1))
#pr_img = pr_img.updateMask(mask)
pr_dict = {}
pr_dict['ag'] = pr_img.select(['ag'], ['prob']).addBands(ee.Image.constant(19).byte())
pr_dict['shrub'] = pr_img.select(['shrub'], ['prob']).addBands(ee.Image.constant(5).byte())
pr_dict['for'] = pr_img.select(['for'], ['prob']).addBands(ee.Image.constant(1).byte())
pr_dict['plant'] = pr_img.select(['plant'], ['prob']).addBands(ee.Image.constant(3).byte())
pr_collection = ee.ImageCollection([pr_dict['for'], pr_dict['plant'], pr_dict['shrub'], pr_dict['ag']])
mosaic = pr_collection.qualityMosaic('prob').select(['constant'], ['elu'])
transition_img = map_dict['olu'].multiply(100).add(mosaic)

transitions = [101, 103, 105, 119, 301, 303, 305, 319, 501, 
               503, 505, 519, 1901, 1903, 1905, 1919]
new_bands = ['constant']
additional_bands = ['constant_' + str(i) for i in np.arange(1,16)]
new_bands.extend(additional_bands)
t_bands = [str(i) for i in transitions]
transition_bands = transition_img.eq(transitions).multiply(ee.Image.pixelArea().divide(10000000))
transition_bands = transition_bands.select(new_bands, t_bands)


def add_change_areas(feature):
    changes = transition_bands.reduceRegion(reducer = ee.Reducer.sum(), 
                                            geometry = feature.geometry(), 
                                            scale = 30, 
                                            maxPixels = 5e9)
    t_dict = {}
    for t in transitions:
        t_dict[str(t)] = changes.get(str(t))
        feature = feature.set(t_dict)
    return feature

out = regions.map(add_change_areas)
task = ee.batch.Export.table.toDrive(collection = out, description = 'Sim_test_' + str(n),  
                                     fileFormat = 'csv', fileNamePrefix = 'sim_test_' + str(n))
task.start()

In [ ]:
regions.getInfo()

In [ ]:
## Investigate regions

band = 'Ran_uniform'
# img = ee.Image(cum_pr_ic.first()).select(band)
runi =  gee_random.runiform_raster()
img = runi(img)
hist = img.reduceRegion(reducer = ee.Reducer.frequencyHistogram(), geometry = sample_region.geometry(), scale = 30)
hist_info = hist.getInfo()
series = pd.Series(hist_info[band])
